First, clone the repository

In [1]:
!git clone https://github.com/vairodp/AstroNet.git

Cloning into 'AstroNet'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (837/837), done.
remote: Compressing objects: 100% (545/545), done.
remote: Total 837 (delta 471), reused 604 (delta 261), pack-reused 0
Receiving objects: 100% (837/837), 35.74 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (471/471), done.


In [2]:
%cd AstroNet

/content/AstroNet


Then install the missing libraries that our code requires:

In [3]:
!pip install tensorflow_addons
!pip install tensorflow-datasets==4.3.0
!pip install imgaug==0.4.0

     |████████████████████████████████| 1.1 MB 11.9 MB/s 
     |████████████████████████████████| 3.9 MB 12.7 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
     |████████████████████████████████| 948 kB 11.7 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


Make sure you're using a GPU in order to get fast train and inference.

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

Finally, run the training code

In [7]:
%cd src

/content/AstroNet/src


In [8]:
from unet import SourceSegmentation
from datasets.convo_ska import ConvoSKA
from callbacks.display_callback import DisplayCallback
from configs.train_config import ITER_PER_EPOCH, NUM_EPOCHS


# Set use_class_weights=False to run the model without weights for the 4 classes
# Set tiny=True to run the smaller version of this model
unet = SourceSegmentation((128,128,1), use_class_weights=True, tiny=False)
unet.model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003, clipvalue=1.0)
checkpoint_filepath = '../checkpoints/unet-best.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='../log')
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10)

unet.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
dataset_train = ConvoSKA(mode='train').get_dataset()
val_data = ConvoSKA(mode='validation').get_dataset()
display_callback = DisplayCallback(val_data)


unet.fit(dataset_train, epochs=NUM_EPOCHS, validation_data=val_data, 
        callbacks=[model_checkpoint_callback, display_callback, tensorboard_callback, early_stop_callback], steps_per_epoch=ITER_PER_EPOCH)


Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

19/70 [=======>......................] - ETA: 23s - loss: 0.0025 - accuracy: 0.6539

KeyboardInterrupt: ignored